In [59]:
import os
# real_data = set([x.strip('.pt') for x in os.listdir("/workspace/human_features/processed")])
# real_data = set([x.strip('.pdb') for x in os.listdir("/workspace/ppi_dataset/pdb_files-negative")])
real_data = set([x.strip('.pt') for x in os.listdir("/workspace/human_features/processed")])
print(len(real_data))
# real_data.update(
#     set([x.strip('.pdb') for x in os.listdir("/workspace/ppi_dataset/pdb_files-positive")])
# )
# print(len(real_data))

7209


In [60]:
list(real_data)[0]

'7PLS'

In [61]:
from copy import deepcopy

positive_labels = [x.strip().split() for x in open('/workspace/ppi_dataset/updated_interactions-POSITIVE.txt', 'r').readlines()]
negative_labels = [x.strip().split() for x in open('/workspace/ppi_dataset/updated_interactions-NEGATIVE-FINAL.txt', 'r').readlines()]

full_data = []
for row in positive_labels:
    row[-1] = float(row[-1])
    full_data.append(row)
for row in negative_labels:
    full_data.append(row + [0.0])

In [62]:
positive_mapping = [x.strip().split() for x in open('/workspace/ppi_dataset/proteinID2PDB-positive_samples.txt', 'r').readlines()]
negative_mapping = [x.strip().split() for x in open('/workspace/ppi_dataset/proteinID2PDB-negative_samples.txt', 'r').readlines()]

mapping = {}
rev_mapping = {}
for k, v in positive_mapping:
    mapping[k] = v
    rev_mapping[v] = k
for k, v in negative_mapping:
    mapping[k] = v
    rev_mapping[v] = k

In [63]:
full_data = [[mapping[x[0]], mapping[x[1]], x[2]] for x in full_data]

In [64]:
filtered_data = []
for row in full_data:
    if row[0] in real_data and row[1] in real_data:
        filtered_data.append(row)

In [65]:
len(filtered_data), len(full_data)

(149497, 171925)

In [66]:
from sklearn.model_selection import train_test_split

train_names, test_names = train_test_split(list(real_data), test_size=0.2)

In [67]:
train_names = set(train_names)
test_names = set(test_names)

In [68]:
train_pairs = []
test_pairs = []
mixed_pairs = []
for row in filtered_data:
    if row[0] in train_names and row[1] in train_names:
        train_pairs.append(row)
    elif row[0] in test_names and row[1] in test_names:
        test_pairs.append(row)
    else:
        mixed_pairs.append(row)

In [69]:
len(train_pairs), len(test_pairs), len(mixed_pairs)

(98396, 5677, 45424)

In [70]:
import numpy as np

def generate_file(pairs, path):
    data = [['', '', row[0], '', '', row[1], row[2]] for row in pairs]
    np.random.shuffle(data)
    data = np.array(data, dtype='<U12')
    np.save(path, data)

In [71]:
generate_file(train_pairs, 'human_features/npy_file_new(human_dataset).npy')
generate_file(test_pairs, 'human_features/npy_file_new(human_dataset)_test.npy')

In [51]:
import os
files = os.listdir('human_features/raw')

In [54]:
'2OZB.pdb' in files

True

In [56]:
pfiles = os.listdir('human_features/processed')
'2OZB.pt' in pfiles

False

In [58]:
len(files), len(pfiles)

(7820, 7209)

In [89]:
negatives = "/workspace/data/ppi_dataset/pdb_negative-dataset.txt"
positives = "/workspace/data/ppi_dataset/positive_PDB-ids.txt"

negatives = [x.strip() for x in open(negatives, "r").readlines() if x.strip() in real_data]
positives = [x.strip() for x in open(positives, "r").readlines() if x.strip() in real_data]

In [90]:
len(negatives), len(positives)

(1918, 6459)

In [91]:
from sklearn.model_selection import train_test_split

negative_train, negative_test = train_test_split(negatives, test_size=0.2, random_state=42)
positive_train, positive_test = train_test_split(positives, test_size=0.2, random_state=42)

In [92]:
def generate_data(els, label):
    data = []
    for el in els:
        for el2 in els:
            if el != el2:
                data.append([el, el2, label])
    return data

negative_train_pairs = generate_data(negative_train, 0)
negative_test_pairs = generate_data(negative_test, 0)
positive_train_pairs = generate_data(positive_train, 1)
positive_test_pairs = generate_data(positive_test, 1)

In [93]:
negative_train_pairs = negative_train_pairs[:100000]
positive_train_pairs = positive_train_pairs[:100000]

negative_test_pairs = negative_test_pairs[:20000]
positive_test_pairs = positive_test_pairs[:20000]

In [94]:
data_train = negative_train_pairs + positive_train_pairs
data_test = negative_test_pairs + positive_test_pairs

data_train = [['', '', row[0], '', '', row[1], row[2]] for row in data_train]
data_test = [['', '', row[0], '', '', row[1], row[2]] for row in data_test]

np.random.shuffle(data_train)
np.random.shuffle(data_test)

data_train = np.array(data_train, dtype='<U12')
data_test = np.array(data_test, dtype='<U12')


np.save("/workspace/human_features/npy_file_new(human_dataset).npy", data_train)
np.save("/workspace/human_features/npy_file_new(human_dataset)_test.npy", data_test)

In [96]:
data_train

array([['', '', '3FIE', ..., '', '2E7M', '1'],
       ['', '', '1DEB', ..., '', '7W3Z', '1'],
       ['', '', '8IP3', ..., '', '2F5H', '1'],
       ...,
       ['', '', '3FIE', ..., '', '4NJ8', '1'],
       ['', '', '1BTN', ..., '', '2L7Z', '0'],
       ['', '', '4PZN', ..., '', '3OGK', '0']], dtype='<U12')

In [101]:
import torch
from torch_geometric.data import Data

file = "/workspace/human_features/processed/4PZN.pt"

def bump(g):
    return Data.from_dict(g.__dict__)

bump(torch.load(file))

Data()

In [56]:
import os
for name in os.listdir("/workspace/human_features/processed"):
    if name.startswith("Q"):
        print(name)


In [72]:
import numpy as np

In [75]:
data = np.load('human_features/npy_file_new(human_dataset).npy')

In [76]:
data

array([['', '', '2MRE', ..., '', '4GEI', '0.335467'],
       ['', '', '2DMV', ..., '', '1LB4', '0.599047'],
       ['', '', '2XU7', ..., '', '1XWW', '0.0'],
       ...,
       ['', '', '1D2S', ..., '', '1D2S', '0.254085'],
       ['', '', '8WQR', ..., '', '5JCO', '0.335467'],
       ['', '', '4JNB', ..., '', '1S3O', '0.0']], dtype='<U12')